In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
sns.set_style('white')

In [ ]:
df = [pd.read_csv(fn, index_col=0) for fn in snakemake.input]
df = pd.concat(df, ignore_index=True)

df["# Variables"] = df.N ** 2 * 2
df['# Constraints'] = df.N ** 2 * 2

In [ ]:
fig, ax = plt.subplots()
color = ['orange', 'teal', 'green']
sns.lineplot(x="# Variables", y='Time [s]', hue='API', 
             data=df, ax=ax, marker='.', legend='full', palette=color)
for i, api in enumerate(df.API.unique()):
    d = df.query('API == @api')
    ax.fill_between(d['# Variables'], d['Time [s]'], 
                           color=color[i], alpha=.2) 
ax.grid(axis='y', linestyle="dotted")
ax.legend(framealpha=1)
fig.tight_layout()
fig.savefig(snakemake.output.time)

In [ ]:
fig, ax = plt.subplots()
color = ['orange', 'teal', 'green']
sns.lineplot(x="# Variables", y='Memory Usage', hue='API', 
             data=df, ax=ax, marker='.', legend='full', palette=color)
for i, api in enumerate(df.API.unique()):
    d = df.query('API == @api')
    ax.fill_between(d['# Variables'], d['Memory Usage'], 
                           color=color[i], alpha=.2) 
ax.grid(axis='y', linestyle="dotted")
ax.legend(framealpha=1)
fig.tight_layout()
fig.savefig(snakemake.output.memory)